In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import seaborn as sns


#Import Google Key
from api_keys import g_key
gmaps.configure(api_key=g_key)


ModuleNotFoundError: No module named 'api_keys'

In [ ]:
g_key



In [ ]:
#Load CSV file as DataFrame
city_data = pd.read_csv("cities.csv")
city_data.head(10)

In [ ]:
#Set Humidity as Weight
humidity = city_data["Humidity"].astype(float)
maxhumidity = humidity.max()

#Set Lat and Lng as Locations
locations = city_data[["Lat", "Lng"]]

In [ ]:
#Plot Map
#fig = gmaps.figure()
# Plost as a Hybrid map
fig = gmaps.figure(map_type="HYBRID")

#Add Heat Layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)

#Show Map
fig

In [ ]:
#Narrow Cities to Reflect Conditions
narrowed_cities = city_data.loc[(city_data["Max Temp"] > 70) & (city_data["Max Temp"] < 80) & (city_data["Cloudiness"] == 0), :]

#Drop Null Values
narrowed_cities = narrowed_cities.dropna(how='any')
narrowed_cities.reset_index(inplace=True)

#Delete Unneccessary Columns
#del narrowed_cities["Unnamed: 0"]
#del narrowed_cities["index"]

#Show DataFrame
narrowed_cities.head(10)


In [ ]:
#Create Empty List for Hotels
hotel_df = []

#Begin For Loop to Find Hotels
for i in range(len(narrowed_cities)):
    lat = narrowed_cities.loc[i]['Lat']
    lng = narrowed_cities.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key 
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel_df.append(jsn['results'][0]['name'])
    except:
        hotel_df.append("")
        
#Create DataFrame        
narrowed_cities["Hotel Name"] = hotel_df

#Drop Null Values
narrowed_cities = narrowed_cities.dropna(how='any')

#Show DataFrame
narrowed_cities.head(10)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_cities.iterrows()]
locations = narrowed_cities[["Lat", "Lng"]]

In [ ]:
markers = gmaps.marker_layer(locations)
fig = gmaps.figure(map_type="HYBRID")
fig.add_layer(markers)

#Display figure
fig